In [ ]:
import pandas as pd
import numpy as np

stock_prices = pd.read_csv('data/stock_prices.csv')
stock_prices['timestamp'] = pd.to_datetime(stock_prices['timestamp'])
stock_prices = stock_prices.set_index('timestamp').sort_index(ascending=True)
shifted_stock_prices = stock_prices.shift(1).add_suffix('_last_close')

column_mapping = {
    'AA': 'AAL',
    'UA': 'UAL',
    'B6': 'JBLU',
    'AS': 'ALK',
    'NK': 'SAVE',
    'WN': 'LUV',
    'DL': 'DAL',
    'HA': 'HA'
}
tickers = ['AAL', 'UAL', 'JBLU', 'ALK', 'SAVE', 'LUV', 'DAL', 'HA']

stock_prices.rename(columns=column_mapping, inplace=True)

# Calculate log prices and add suffix '_log_price'
stock_price_log = stock_prices.apply(np.log).add_suffix('_log_price')

# Define the rolling windows
windows = [7, 14, 30, 90]
for window in windows:
    shifted_prices_window = stock_prices.shift(window, freq='D').add_suffix(f'_{window}_days_ago')
    shifted_stock_prices = pd.merge(shifted_stock_prices, shifted_prices_window, how='left', left_index=True, right_index=True)
    
# Calculate rolling averages for each column and window size
for col in tickers:
    for window in windows:
        new_col_name = f'{col}_rolling_{window}_day_avg'
        stock_prices[new_col_name] = stock_prices[col].rolling(window).mean()




# Combine the original DataFrame, shifted DataFrame, and log prices DataFrame
main_stock_price_df = pd.concat([stock_prices, shifted_stock_prices, stock_price_log], axis=1)

for col in tickers:
    for window in [7, 14, 30, 90]:
        shifted_col = f'{col}_{window}_days_ago'
        new_col_name = f'{col}_pct_change_{window}_days'
        main_stock_price_df[new_col_name] = (main_stock_price_df[col] - main_stock_price_df[shifted_col]) / main_stock_price_df[shifted_col]

pd.set_option('display.max_columns', 300)
display(main_stock_price_df.head())